Copyright **`(c)`** 2023 Antonio Ferrigno `<s316467@studenti.polito.it>`  
[`https://github.com/s316467/Computational-Intelligence-23-24`](https://github.com/s316467/Computational-Intelligence-23-24)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/s316467/Computational-Intelligence-23-24/LICENSE.md) for details. 

In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
from statistics import mean
from numba import njit

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_matrix((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
@njit
def fitness(solution, sets, num_points):
    covered_points = np.zeros(num_points, dtype=np.bool_)
    for i in solution:
        covered_points |= sets[i, :]
    return np.sum(covered_points) - len(solution)

def greedy_algorithm(sets, num_points):
    sets_np = sets.todense().astype(np.bool_)
    covered_points = np.zeros(num_points, dtype=np.bool_)
    selected_sets = []
    fitness_calls = 0
    while np.sum(covered_points) < num_points:
        best_set = -1
        best_cover = -1
        for i in range(sets_np.shape[0]):
            if i not in selected_sets:
                new_cover = np.sum(np.logical_and(np.logical_not(covered_points), sets_np[i, :]))
                if new_cover > 0:  # Consider only sets that add new points
                    if new_cover > best_cover:
                        best_cover = new_cover
                        best_set = i
        covered_points |= sets_np[best_set, :].A1  # Convert to 1D array before the operation
        selected_sets.append(best_set)
        fitness_calls += 1  # Count the fitness calls
    return selected_sets, fitness_calls


def plot_solution(sets, solution, num_points):
    fig, ax = plt.subplots(figsize=(10, 8))
    cmap = plt.cm.Blues
    cax = ax.matshow(sets.todense(), cmap=cmap)
    for i in solution:
        ax.add_patch(plt.Rectangle((0, i), num_points, 1, fill=True, color='red', alpha=0.5))
    plt.xlabel('Points')
    plt.ylabel('Sets')
    plt.title('Set Covering Problem')
    plt.show()

In [4]:
num_points_list = [100, 1_000, 5_000]
density_list = [0.3, 0.7]

results = {}
plot_data = []
best_fitness = float('inf')
resets = 0

for num_points in num_points_list:
    results[num_points] = {}
    for density in density_list:
        sets = make_set_covering_problem(num_points, num_points, density)
        best_solution = None
        best_fitness = float('-inf')
        fitness_calls_list = []
        resets = 0
        for _ in range(10):
            solution, fitness_calls = greedy_algorithm(sets, num_points)
            fitness_value = fitness(solution, sets.todense().astype(np.bool_), num_points)
            fitness_calls_list.append(fitness_calls)
            if fitness_value > best_fitness:
                best_fitness = fitness_value
                best_solution = solution
                resets += 1
        avg_fitness_calls = mean(fitness_calls_list)
        print(f"num_points: {num_points}, density: {density}, best solution size: {len(best_solution)}, best fitness: {best_fitness}, average fitness calls: {avg_fitness_calls}, resets: {resets}")
        results[num_points][density] = {
            'best_solution': best_solution,
            'best_fitness': best_fitness,
            'avg_fitness_calls': avg_fitness_calls,
            'resets': resets
        }
        plot_data.append((sets, best_solution, num_points))

# Plot all solutions
for sets, best_solution, num_points in plot_data:
    plot_solution(sets, best_solution, num_points)

ValueError: non-broadcastable output operand with shape (100,) doesn't match the broadcast shape (1,100)